In [ ]:
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
'''
filepath = 'D:\\mlinternship\\iitgdata\\feb1-15\\HW_2_part_1\\feb-01-2022.xlsx'
df = pd.read_excel (filepath, header = 3)
df
'''


1

In [ ]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)

    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)

            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)

            # Append the dataframe to the list
            df_list.append(df)

In [ ]:
'''
# adding the data from march (1-15) 2022 to the dataframe
folder = 'D:\\mlinternship\\iitgdata\\mar1-15'

march1_DF = []
for filename in os.listdir(folder):
    if filename.endswith('.xlsx'):
        file = os.path.join(folder, filename)
        march1_DF.append(pd.read_excel(file, header = 3))
#df = pd.concat(DF, ignore_index=True)
march1_DF




# adding the data from march (16-31) 2022 to the dataframe
folder = 'D:\\mlinternship\\iitgdata\\mar16-31\\EE677_HW2_A_200102042'

march2_DF = []
for filename in os.listdir(folder):
    if filename.endswith('.xlsx'):
        file = os.path.join(folder, filename)
        march2_DF.append(pd.read_excel(file, header = 3))
#df = pd.concat(DF, ignore_index=True)
march2_DF



#preparing the dataframe with all the data from april 16-30
folder = 'D:\\mlinternship\\iitgdata\\Apr16to30-2022-data 2\\Apr16to30-2022-data'

april2_DF = []
for filename in os.listdir(folder):
    if filename.endswith('.xlsx'):
        file = os.path.join(folder, filename)
        april2_DF.append(pd.read_excel(file, header = 3))
#df = pd.concat(DF, ignore_index=True)
april2_DF




#preparing the dataframe with all the data from june 1-15
folder = 'D:\\mlinternship\\iitgdata\\june 1-15'

june1_DF = []
for filename in os.listdir(folder):
    if filename.endswith('.xlsx'):
        file = os.path.join(folder, filename)
        june1_DF.append(pd.read_excel(file, header = 3))
#df = pd.concat(DF, ignore_index=True)
june1_DF = pd.concat(june1_DF)


temporary_csv_path  = 'D:\\mlinternship\\iitgdata\\june 1-15\\temporary\\csvfile2'
csvfile = june1_DF.to_csv(temporary_csv_path)
june1_DF = pd.read_csv(temporary_csv_path)
# for some reason data from june 2nd 6 am onwards is behaving strangely(6 am being read as 5:59:59.9)
june1_DF['Time'] = pd.to_datetime(june1_DF['Time'])
june1_DF['Time'] = june1_DF['Time'].round('min')
june1_DF
'''

2

In [ ]:
power_df = pd.concat(df_list, ignore_index=True)
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
# drop the rest of the columns that are not required and set index to Time
power_df = power_df[['Time', 'MW']]


power_df

In [ ]:
power_df.dropna(subset=['Time'], inplace=True)
power_df.set_index('Time', inplace=True)
#converting the time column to the UTC format
power_df.index = pd.to_datetime(power_df.index)

# Convert the datetime objects to UTC timezone
power_df.index = power_df.index.tz_localize(pytz.timezone('UTC'))
# Now, 'Time' column is in UTC timezone format
power_df

In [ ]:
# read the temperature data csv
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
#drop the unwanted columns and change column name
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]
temperature_df


In [ ]:
#resetting the index to prepare it to join with the power dataframe
temperature_df.set_index('Time', inplace=True)
temperature_df

In [ ]:
from datetime import timedelta

#changing the time column to a common UTC format
temperature_df.index = pd.to_datetime(temperature_df.index)

# Convert the datetime objects to UTC timezone
temperature_df.index = temperature_df.index.tz_localize(pytz.timezone('UTC'))
temperature_df.index = pd.DatetimeIndex(temperature_df.index) + timedelta(hours=5,minutes=30)
# Now, 'Time' column is in UTC timezone format



plt.figure(figsize=(10, 5))
plt.ion()
plt.plot(temperature_df.index, temperature_df['temperature'], marker='o', linestyle='-', color='r', label='MW')

plt.xlabel('Time')

plt.xticks(rotation=45)

ax = plt.gca()
ax.set_facecolor('white')
ax.set_title(' temperature vs time')
plt.legend()
plt.tight_layout()

plt.show()


In [ ]:
temperature_df

In [ ]:
# joining the two dataframes such that the temperature data is only taken if there exists a reading in the power data dataframe
df = pd.merge(power_df, temperature_df, on='Time', how='left')

DF = df.copy()
#drop all the rows where NaN
df.dropna(inplace = True)
df = df.sort_index(ascending=True)
df



3

In [ ]:

df['temperature'] = pd.to_numeric(df['temperature'], errors='coerce')

# Rest of your code for CDH calculation
TcoolStPt = 31
CDH = df['temperature'] - TcoolStPt
CDH.clip(lower=0, inplace=True)
CDH = pd.DataFrame(data=CDH.values, columns=['CDH'], index=df.index)

CDH.index = df.index

# Perform the left join without creating an extra column
df.reset_index(drop=False, inplace=True)

# Reset the index of CDH DataFrame
CDH.reset_index(drop=False, inplace=True)


df = pd.concat([df, CDH['CDH']], axis=1)
df.set_index('Time', inplace = True)

df

4

In [ ]:
df['MW'] = df['MW'].astype(str)

# Remove commas from 'CDH' column and convert to numeric
df['MW'] = pd.to_numeric(df['MW'].str.replace(',', '.'), errors='coerce')

# Check if there are any other non-numeric values in the 'CDH' column
non_numeric_cdh = df['MW'][pd.to_numeric(df['MW'], errors='coerce').isnull()]

CDHmask = (df['CDH'] > 0)
f,ax = plt.subplots(nrows=1, figsize = (20,8))
_=ax.scatter(df['CDH'][CDHmask],df['MW'][CDHmask],label='vs. cooling degree hour')
_ = ax.set_title('Scatter Plot: MW vs. Cooling Degree Hour')
_=ax.legend()


In [ ]:
plt.ion()
df.index = pd.to_datetime(df.index)
f,ax = plt.subplots(figsize=(30,6))
plt.plot(df.index,df['temperature'])
plt.scatter(df.index,df['temperature'],color='red')
ax.set_title('Temperature Over Time')

In [ ]:
plt.ion()
df['MW'] = pd.to_numeric(df['MW'], errors = 'coerce')
f,ax = plt.subplots(nrows = 1, figsize=(5,5))
#plt.plot(df['temperature'], df['MW'])
plt.scatter(df['temperature'], df['MW'],color='red')
ax.set_title('MW vs Temperature')

5

In [ ]:
df = df[df['MW'] <= 20]


In [ ]:
#temporary_df = df.dropna(subset=['temperature', 'MW'])

start_time = '2002-06-01 00:00:00+00:00'
end_time  = '2024-10-15 23:00:00+00:00'

mask = (df.index >= start_time) & (df.index <= end_time)
time_mask = df[mask]
time_mask = time_mask.dropna(subset=['temperature', 'MW'])

# Plot 'MW' against the index of time_mask
# Create a scatter plot for y_train against x_train
power_normalized = preprocessing.normalize([time_mask['MW']]).reshape(-1,1)
temperature_normalized = preprocessing.normalize([time_mask['temperature']]).reshape(-1,1)




plt.figure(figsize=(10, 5))
plt.ion()
plt.plot(time_mask.index, time_mask['MW'], marker='o', linestyle='-', color='r', label='MW')
plt.plot(time_mask.index, time_mask['temperature'], marker='o', linestyle='-', color='y', label='temp')
plt.xlabel('Time')

plt.xticks(rotation=45)

ax = plt.gca()
ax.set_facecolor('white')
ax.set_title('power and temperature vs time (not normalized)')
plt.legend()
plt.tight_layout()

plt.show()


plt.figure(figsize=(10, 5))
plt.ion()
plt.plot(time_mask.index, power_normalized, marker='o', linestyle='-', color='r', label='MW')
plt.plot(time_mask.index, temperature_normalized, marker='o', linestyle='-', color='y', label='temp')
plt.xlabel('Time')

plt.xticks(rotation=45)

ax = plt.gca()
ax.set_facecolor('white')
ax.set_title('power and temperature vs time (normalized)')
plt.legend()
plt.tight_layout()

plt.show()

6

In [ ]:
numOmegas = 24 * 7

df = df.reset_index(drop=False)

num_of_rows = df.shape[0]
omegas = np.zeros((num_of_rows, numOmegas))  # Assuming numOmegas columns for omegas
concatenated_data = np.concatenate((df, omegas), axis=1)


column_names = ['Time', 'MW', 'Temperature', 'CDH']
for i in range(1, numOmegas + 1,1):
    column_names.append('omega' + str(i))

df = pd.DataFrame(concatenated_data, columns=column_names)
df['Time'] = pd.to_datetime(df['Time'])  # Convert 'Time' column to datetime if needed
df = df.set_index('Time')

for i in range(0,num_of_rows):
        datetime = df.index[i]
        hourOfWeekIndex = int(datetime.dayofweek*24+(datetime.hour+1))
        x = np.zeros((1,numOmegas))
        x[0,hourOfWeekIndex-1]=1
        omegas[i,:]=x


df.iloc[:,3:]=omegas
df


In [ ]:

from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import itertools


7

In [ ]:

df.reset_index(inplace=True)
start_time = pd.Timestamp('2022-07-14 00:00:00+00:00')
end_time = pd.Timestamp('2022-08-30 23:00:00+00:00')
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)

filtered_df = df[training_mask]

filtered_df = filtered_df.dropna(subset=['CDH'])

filtered_df = filtered_df.dropna(subset=['MW'])
# Now, filtered_df contains only the rows where 'Temperature' is not NaN

# Extract the required columns for training and testing
x_train = filtered_df.loc[:, 'CDH']
y_train = filtered_df.loc[:, 'MW']
t_train = filtered_df['Time']  # Time for training data
testing_end_time = pd.Timestamp('2022-10-15 23:00:00+00:00')
# For testing data, you might want to use data after end_time
testing_mask = (df['Time'] > end_time) & (df['Time'] <= testing_end_time) & (df['CDH'] > 0)
testing_df = df[testing_mask]
testing_df = testing_df.dropna(subset=['CDH'])
testing_df = testing_df.dropna(subset=['MW'])
x_test = testing_df.loc[:, 'CDH']
y_test = testing_df.loc[:, 'MW']
t_test = testing_df['Time']  # Time for testing data


In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
'''f,ax = plt.subplots(figsize=(30,6))

#plt.plot(df['temperature'], df['MW'])
plt.scatter(x_test,y_test,color='red')
ax.set_title('x test vs y test')
'''

In [ ]:
'''
x_train = x_train.values.reshape(-1, 1)
x_test = x_test.values.reshape(-1, 1)
rmses = []
degrees = np.arange(1, 10)
min_rmse, min_deg = 1e10, 0

for deg in degrees:

    # Train features
    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    x_poly_train = poly_features.fit_transform(x_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly_train, y_train)

    # Compare with test data
    x_poly_test = poly_features.fit_transform(x_test)
    poly_predict = poly_reg.predict(x_poly_test)
    poly_mse = mean_squared_error(y_test, poly_predict)
    poly_rmse = np.sqrt(poly_mse)
    rmses.append(poly_rmse)

    # Cross-validation of degree
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg

# Plot and present results
print('Best degree {} with RMSE {}'.format(min_deg, min_rmse))

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(degrees, rmses)
ax.set_yscale('log')
ax.set_xlabel('Degree')
ax.set_ylabel('RMSE')
'''


8

In [ ]:
x_train = pd.to_numeric(x_train, errors='coerce')
x_train = x_train.values.reshape(-1, 1)
y_train = np.array(y_train)
# Linear regression
poly_reg = LinearRegression()
poly_reg.fit(x_train, y_train)

predictions = poly_reg.predict(x_train)
coef_cooling = poly_reg.coef_
CDHmodelScore = poly_reg.score(x_train,y_train)

print("score of the model is ", CDHmodelScore)

plt.figure(figsize=(10, 5))

plt.scatter(x_train, y_train, color='blue', label='Actual MW')

# Create a line plot for predictions against x_train
plt.scatter(x_train, predictions, color='red', label='Predicted MW')

# Set labels and title
plt.xlabel('x_train')
plt.ylabel('MW')
plt.title('Actual vs Predicted MW (using only CDH values to predict)')
plt.legend()  # Show legend

# Show the plot
plt.show()


In [ ]:
coef_cooling

9

In [ ]:
X = df.loc[:,'Temperature':'omega168']
X

In [ ]:

training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)

#Linear regression for the rest of the model

x_behavior = df.loc[:,'omega1':'omega168'][training_mask]
y = pd.DataFrame(df['MW'][training_mask])

yhat_c = (coef_cooling*df.loc[:,'CDH'][training_mask].values).reshape(-1,1)

y_behavior = y - (yhat_c)

# Drop rows where y_behavior has NaN values
y_behavior = y_behavior.dropna()

# Get the index of non-null values in y_behavior and use it to filter x_behavior
non_null_index = y_behavior.index
x_behavior_filtered = x_behavior.loc[non_null_index]
x_behavior= x_behavior_filtered



time = df['Time'][training_mask]

## ------ fit the behavior model -----
model = LinearRegression(fit_intercept=False, positive=True)
model.verbose=False
model.fit(x_behavior.values,y_behavior.values)
#model.coef_,model.intercept_
behavior_modelScore = model.score(x_behavior.values,y_behavior.values)
print('score of the behavior model is '+str(behavior_modelScore))
#predict:
yhat_behavior = model.predict(x_behavior.values)



# Combine time index and y_behavior values into a new DataFrame
predicted_data = pd.DataFrame({'Time': time, 'Predicted MW': yhat_behavior.reshape(-1)})

# Plotting
plt.figure(figsize=(10, 5))
plt.scatter(predicted_data['Time'], predicted_data['Predicted MW'], color='red', label='Predicted MW')
plt.scatter(predicted_data['Time'], y_behavior.values, color='blue', label='Actual MW')

# Set labels and title
plt.xlabel('Time')
plt.ylabel('MW')
plt.title('Actual vs Predicted MW (using behavior model)')
plt.legend()
plt.show()

'''

plt.figure(figsize=(10, 5))

plt.scatter(time, y_behavior.values, color='blue', label='Actual MW')
plt.scatter(time, yhat_behavior, color='red', label='Predicted MW')

# Set labels and title
plt.xlabel('time')
plt.ylabel('MW')
plt.title('Actual vs Predicted MW (using behavior model)')
plt.legend()
plt.show()
'''
#compute the normalized 1-norm based prediction error
# -- in sample
'''
junk = yhat_behavior - y_behavior.loc[:].to_numpy().reshape(-1,1)
e_behavior = np.linalg.norm(junk,ord=1)/np.linalg.norm(y_behavior,ord=1)
'''
#save for later
model_behavior = model


In [ ]:
model_behavior.coef_

In [ ]:
time

10

In [ ]:

""" dangerous stuff being done here: manually construct a ful LR model"""
# the dataframe holding X has [T_heat, T_cool, omegas....]
c = np.concatenate((np.array([coef_cooling]).reshape(1,-1),
                    model_behavior.coef_),axis=1)
model_1 = LinearRegression(fit_intercept=False)
model_1.coef_ = c
model_1.intercept_ = 0

# predict using the complete model. We can also add the components already
# computedm btu this is cleaner (ideally should compare the two!)

X = df.loc[:,'CDH':'omega168']
y = pd.DataFrame(df['MW'])
time = df.loc[:,'Time']

y = y.dropna()
X = X.loc[y.index]
X = X.dropna()
y = y.loc[X.index]
# Update time based on filtered y
time = df.loc[y.index, 'Time']
yhat = model_1.predict(X.values)
fullmodelScore = model_1.score(X,y)
print ('score for constructed full model: ', fullmodelScore)
"""
#compute the normalized 1-norm based prediction error
# -- in sample
junk = yhat - y.loc[:].to_numpy().reshape(-1,1)
e_model1 = np.linalg.norm(junk,ord=1)/np.linalg.norm(y,ord=1)
"""

# prediction for the full model
# yhat = yhat_bahavior+(yhat_c+yhat_h)
# against time
fig,(ax1) = plt.subplots(nrows=1,figsize=(10,9))
_=ax1.plot(time,y,label='meas')
_=ax1.plot(time,yhat,label='pred: behavior+heat/cool')
ax1.set_title('measured vs predicted data (full constructed model)')
_=ax1.legend()

'''
print('+++++ for model 1, constructed with heat/cool/behavior done separately and then joined++++:')
print('1 norm of the normalized pred error with the training data ='+str(e_model1))
'''

In [ ]:
y = y.dropna()
X = X.loc[y.index]

# Update time based on filtered y
time = df.loc[y.index, 'Time']
has_nan_X = X.isna().any().any()

# For Series y
has_nan_y = y.isna().any()

print("X contains NaN values: ", has_nan_X)
print("y contains NaN values: ", has_nan_y)
# Drop NaN values from y and corresponding rows from X


In [ ]:
'''
## ------- model version 2------
## Full model in one shot, like in the past

## ------ fit the model -----
model_2 = LinearRegression(fit_intercept=False)
model_2.verbose=False
model_2.fit(X.values,y.values)

#model.coef_,model.intercept_
modelScore = model_2.score(X.values,y.values)
print('score of the full, one shot, model is '+str(modelScore))
#predict:
yhat = model_2.predict(X.values)
filtered_cdh = df.loc[y.index, 'CDH']
#compute the normalized 1-norm based prediction error
# -- in sample
junk = yhat - y.loc[:].to_numpy().reshape(-1,1)
e_model2 = np.linalg.norm(junk,ord=1)/np.linalg.norm(y,ord=1)

# prediction for the full model
# against time
fig,(ax1,ax2) = plt.subplots(nrows=2,figsize=(10,9))
ax1.set_title(label='Results for the one-shot model')
_=ax1.plot(time,y,label='meas')
_=ax1.plot(time,yhat,label='pred')
_=ax1.legend()
#against tcool and t heat to see if the slope is better?
_=ax2.scatter(filtered_cdh,y,label='demand vs. CDH')
_=ax2.scatter(filtered_cdh,yhat,label='pred vs. CDH')
_=ax2.legend()


print('+++++ for model 2, constructed in one shot')
print('1 norm of the normalized pred error with the training data ='+str(e_model2))
'''
'''
print('------ the coef for Theat and T-cool for the one-shot model = ')
c = model_2.coef_

print('alpha - cool = '+str(c))
print('------ the coef for Theat and T-cool for the one at a time model = ')

print('alpha - cool = '+str(coef_cooling))
'''

In [ ]:
me_array = np.array(filtered_df['MW'], dtype='float64')
temp_array = np.array(filtered_df['Temperature'], dtype='float64')
correlation_coefficient = np.corrcoef(me_array, temp_array)[0, 1]
print(correlation_coefficient)

In [ ]:
#------------------ predict for the whole data set:
XAll=df.loc[:,'CDH':'omega168']
yAll=pd.DataFrame(df.loc[:,'MW'])
timeAll=df['Time']#Series object



def f_getYhatDataFrame(df,model):
    #prediction by model 1
    XAll=df.loc[:,'CDH':'omega168']
    yAll=pd.DataFrame(df.loc[:,'MW'])
    yAll = yAll.dropna()
    XAll = XAll.loc[y.index]

    # Update time based on filtered y
    timeAll = df.loc[y.index, 'Time']

    yAllhat = pd.DataFrame(model_1.predict(XAll.values),columns=['MW'])

    scAll = model.score(XAll.values,yAll.values)

    cs = model.coef_



    c = cs
    yy = df['CDH'] * pd.Series(c.reshape(-1,))
    yAllhat_cool = pd.DataFrame(yy.values,columns=['MW'])

    #save predictions from the fitted model
    XAllNoTempr=XAll.copy()

    XAllNoTempr['CDH'] = np.zeros((XAllNoTempr.shape[0],1))
    yAll_hat_NoTempr = pd.DataFrame(model.predict(XAllNoTempr.values),columns=['MW'])

    junk = [df[['Time','MW']],yAllhat,yAllhat_cool,yAll_hat_NoTempr,df[['CDH']]]
    dfOut = pd.concat(junk,axis=1)
    return dfOut,scAll


df_yhatEtc_model1,sc1 = f_getYhatDataFrame(df,model_1)
print('score for prediction of all data with model 1 is = '+str(sc1))
#df_yhatEtc_model1.head()
#-------- repeat for model 2 ----
'''
df_yhatEtc_model2,sc2 = f_getYhatDataFrame(df,model_2)
print('score for prediction of all data with model 2 is = '+str(sc2))
df_yhatEtc_model2.head()
'''

In [ ]:
'''printfigs=1
years = ('2018','2019','2020')
for year in years:
    t1 = f_time2Eastern(firstMondayMarch[year].iloc[0])
    t2 = t1+datetime.timedelta(days=30)
    mask = (df_yhatEtc_model1['DateTime']>=t1)&(df_yhatEtc_model1['DateTime']<=t2)

    fig,ax = plt.subplots(ncols=1, figsize=(20,5))
    plt.subplots_adjust(bottom=.25)

    _=ax.plot(df_yhatEtc_model1['DateTime'][mask],df_yhatEtc_model1['Demand (MWh)'][mask],color='blue',label='measured')
    _=ax.plot(df_yhatEtc_model1['DateTime'][mask],df_yhatEtc_model1['Demand-pred (MWh)'][mask],color='magenta',linestyle='--',label='predicted')
    #_=ax.plot(df_yhatEtc_model2['DateTime'][mask],df_yhatEtc_model2['Demand-pred (MWh)'][mask],color='brown',linestyle='--',label='predicted, model 2')
    _=plt.legend()
    _=ax.set_ylim(125, 350)
    ax.set_ylabel('Hourly energy use (MWh)')
    locator = mpl.dates.AutoDateLocator(minticks=7, maxticks=12)
    #formatter = mpl.dates.ConciseDateFormatter(locator)
    formatter = mpl.dates.DateFormatter('%a %d\n %m-%y')
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    ax.grid()
    ax.set_xlim(t1,t2)
    _=ax.legend()
    sns.set_context("paper", font_scale=2.5, rc={"lines.linewidth": 1.5})

    bbox_props = dict(boxstyle="round", fc="w", ec="0.5", facecolor="0.1", alpha=0.5)
    ax.text(t1+datetime.timedelta(days=15), 325, year, ha="center", va="center", size=20,
        bbox=bbox_props)

    figname = 'demandPredHourly_'+year+'March_fit2019_'+TmprModel+'.pdf'
    if printfigs:
        fig.savefig('../Figures/forPaper/fromRegression/'+figname)
'''

In [ ]:
new_df = df.dropna()


In [ ]:
mw_array = np.array(new_df['MW'], dtype='float64')
temp_array = np.array(new_df['CDH'], dtype='float64')
correlation_coefficient = np.corrcoef(mw_array, temp_array)[0, 1]
print(correlation_coefficient)

In [ ]:
me_array = np.array(new_df['MW'], dtype='float64')
temp_array = np.array(new_df['Temperature'], dtype='float64')
correlation_coefficient = np.corrcoef(me_array, temp_array)[0, 1]
print(correlation_coefficient)

In [ ]:
df['Time'] = pd.to_datetime(df['Time'])

hours = df['Time'].dt.hour

df_12am_to_6am = df[(hours >= 0) & (hours <= 6)]

df_excluding_12am_to_6am = df[(hours < 0) | (hours > 6)]
df_excluding_12am_to_6am.dropna(inplace= True)

df_12am_to_6am.dropna(inplace=True)


In [ ]:
me_array = np.array(df_12am_to_6am['MW'], dtype='float64')
temp_array = np.array(df_12am_to_6am['Temperature'], dtype='float64')
correlation_coefficient = np.corrcoef(me_array, temp_array)[0, 1]
print('between 12am to 6am: ',correlation_coefficient)

In [ ]:
me_array = np.array(df_excluding_12am_to_6am['MW'], dtype='float64')
temp_array = np.array(df_excluding_12am_to_6am['Temperature'], dtype='float64')
correlation_coefficient = np.corrcoef(me_array, temp_array)[0, 1]
print('excluding 12am to 6am: ',correlation_coefficient)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_12am_to_6am.index, df_12am_to_6am['MW'], marker='o', linestyle='-', color='b')
plt.xlabel('Time')
plt.ylabel('MW')
plt.title('MW vs Time (12:00 AM to 6:00 AM)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_excluding_12am_to_6am.index, df_excluding_12am_to_6am['MW'], marker='o', linestyle='-', color='g')
plt.xlabel('Time')
plt.ylabel('MW')
plt.title('MW vs Time (excluding 12:00 AM to 6:00 AM)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()